# Fourier Flows (Casseau Ch. 3.2.1)

This notebook reproduces the first two Fourier-flow verification cases from Casseau (pure N2 and homogeneous air).
We use the current diffusion model and the new Maxwell-slip / Smoluchowski-jump wall BC.

Notes:
- Case 1: pure N2 (Knov = 0.1).
- Case 2a: homogeneous air (0.5 N2 / 0.5 O2), Knov = 0.002.
- Case 2b: homogeneous air, Knov = 0.1.

You can paste digitized Casseau datapoints into the reference arrays below to overlay in the plots.


In [31]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import fourier_flows_helpers as ff_helpers

import plotly.io as pio
pio.templates.default = "plotly_white"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Settings
# Number densities are in 1/m^3 (Casseau Table 3.3 values are 1e19 m^-3)

CASE1_TR_TB_PATH = (
    ff_helpers.find_repo_root()
    / "experiments"
    / "fourier_flows"
    / "casseau_figure_3_8_case_1_tr_tb.csv"
)
CASE1_TR_TB_REFERENCES = ff_helpers.load_casseau_tr_tb_reference(CASE1_TR_TB_PATH)

CASE_SETTINGS = {
    "case1": {
        "enabled": True,
        "case_name": "case1",
        "species_names": ("N2",),
        "n_by_species": [2.086e19],
        "run_kwargs": {
            "nx": 1,
            "ny": 101,
            "Lx": 1.0,
            "H": 1.0,
            "U": 300.0,
            "Tw_bottom": 2000.0,
            "Tw_top": 3000.0,
            "T_init": 2500.0,
            "t_final": 5e-3,
            "save_interval": 10,
        },
        "references": CASE1_TR_TB_REFERENCES,
    },
    "case2a": {
        "enabled": False,
        "case_name": "case2a",
        "species_names": ("N2", "O2"),
        "n_by_species": [52.15e19, 52.15e19],
        "run_kwargs": {
            "nx": 1,
            "ny": 51,
            "Lx": 1.0,
            "H": 1.0,
            "U": 300.0,
            "Tw_bottom": 2000.0,
            "Tw_top": 3000.0,
            "T_init": 2500.0,
            "t_final": 5e-5,
            "save_interval": 10,
        },
        "references": [],
    },
    "case2b": {
        "enabled": False,
        "case_name": "case2b",
        "species_names": ("N2", "O2"),
        "n_by_species": [1.043e19, 1.043e19],
        "run_kwargs": {
            "nx": 1,
            "ny": 51,
            "Lx": 1.0,
            "H": 1.0,
            "U": 300.0,
            "Tw_bottom": 2000.0,
            "Tw_top": 3000.0,
            "T_init": 2500.0,
            "t_final": 5e-5,
            "save_interval": 10,
        },
        "references": [],
    },
}

PROFILE_SETTINGS = {
    "x_target": None,
}

PLOT_SETTINGS = {
    "figsize": (10, 4),
"sharey": True,
}


## Case 1: Pure N2 (Kn = 0.1)


In [34]:
case1 = None
case1_settings = CASE_SETTINGS["case1"]
if case1_settings["enabled"]:
    case1 = ff_helpers.run_case(
        case1_settings["species_names"],
        case1_settings["n_by_species"],
        case1_settings["case_name"],
        **case1_settings["run_kwargs"],
    )


In [36]:
# cases = [c for c in (case1, case2a, case2b) if c is not None]
cases = [case1]

N_TIME_SAMPLES = 10

profiles_by_time = {}
for case in cases:
    U_hist = case["U_hist"]
    t_hist = np.asarray(case["t_hist"])
    n_steps = len(U_hist)
    if n_steps == 0:
        continue
    sample_count = min(N_TIME_SAMPLES, n_steps)
    indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
    indices = np.unique(indices)

    samples = []
    for idx in indices:
        y, Tn, Tv = ff_helpers.extract_profile_from_U(
            case, U_hist[idx], x_target=PROFILE_SETTINGS["x_target"]
        )
        samples.append((idx, float(t_hist[idx]), y, Tn, Tv))
    profiles_by_time[case["name"]] = samples

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=PLOT_SETTINGS["sharey"],
    subplot_titles=("Normalized T_tr", "Normalized T_v"),
)

# T_tr profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    for idx, t, y, Tn, _Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tn,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=bool(is_final),
                legend="legend",
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1),
                customdata=np.full_like(y, t, dtype=float),
                hovertemplate=(
                    "t=%{customdata:.2e}s<" "br>T_tr/T_b0=%{x:.4f}<" "br>y/H=%{y:.4f}<extra></extra>"
                ),
            ),
            row=1,
            col=1,
        )
    for ref in CASE_SETTINGS[name]["references"]:
        y_ref = ref.get("y_over_H", np.array([]))
        x_ref = ref.get("Ttr_over_Tb0", np.array([]))
        if y_ref.size and x_ref.size:
            label = ref.get("label", name)
            fig.add_trace(
                go.Scatter(
                    x=x_ref,
                    y=y_ref,
                    mode="markers",
                    name=f"Casseau {label}",
                    legendgroup=f"casseau-{label}",
                    showlegend=True,
                    legend="legend",
                    marker=dict(size=6),
                ),
                row=1,
                col=1,
            )

# T_v profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    for idx, t, y, _Tn, Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tv,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=bool(is_final),
                legend="legend2",
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1),
                customdata=np.full_like(y, t, dtype=float),
                hovertemplate=(
                    "t=%{customdata:.2e}s<" "br>T_v/T_b0=%{x:.4f}<" "br>y/H=%{y:.4f}<extra></extra>"
                ),
            ),
            row=1,
            col=2,
        )

fig.update_xaxes(title_text="T_tr / T_b0", row=1, col=1, range=[0.98, 1.52])
fig.update_xaxes(title_text="T_v / T_b0", row=1, col=2, range=[0.98, 1.52])
fig.update_yaxes(title_text="y / H", row=1, col=1)

fig.update_layout(
    width=1000,
    height=500,
    legend_title_text="Cases",
    legend=dict(
        x=0.25,
        y=-0.2,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    legend2=dict(
        x=0.75,
        y=-0.2,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
)

fig.show()


## Case 2a: Homogeneous Air (Kn = 0.002)


In [ ]:
case2a = None
case2a_settings = CASE_SETTINGS["case2a"]
if case2a_settings["enabled"]:
    case2a = ff_helpers.run_case(
        case2a_settings["species_names"],
        case2a_settings["n_by_species"],
        case2a_settings["case_name"],
        **case2a_settings["run_kwargs"],
    )


## Case 2b: Homogeneous Air (Kn = 0.1)


In [ ]:
case2b = None
case2b_settings = CASE_SETTINGS["case2b"]
if case2b_settings["enabled"]:
    case2b = ff_helpers.run_case(
        case2b_settings["species_names"],
        case2b_settings["n_by_species"],
        case2b_settings["case_name"],
        **case2b_settings["run_kwargs"],
    )


## Post-Processing


In [ ]:
# cases = [c for c in (case1, case2a, case2b) if c is not None]

# profiles = {}
# for case in cases:
#     profiles[case["name"]] = ff_helpers.extract_profile(
#         case, x_target=PROFILE_SETTINGS["x_target"]
#     )

# fig = make_subplots(
#     rows=1,
#     cols=2,
#     shared_yaxes=PLOT_SETTINGS["sharey"],
#     subplot_titles=("Normalized T_tr", "Normalized T_v"),
# )

# # T_tr profiles
# for name, (y, Tn, _Tv) in profiles.items():
#     fig.add_trace(
#         go.Scatter(
#             x=Tn,
#             y=y,
#             mode="lines",
#             name=f"{name}",
#             legendgroup=name,
#             showlegend=True,
#         ),
#         row=1,
#         col=1,
#     )
#     for ref in CASE_SETTINGS[name]["references"]:
#         y_ref = ref.get("y_over_H", np.array([]))
#         x_ref = ref.get("Ttr_over_Tb0", np.array([]))
#         if y_ref.size and x_ref.size:
#             label = ref.get("label", name)
#             fig.add_trace(
#                 go.Scatter(
#                     x=x_ref,
#                     y=y_ref,
#                     mode="markers",
#                     name=f"Casseau {label}",
#                     legendgroup=f"casseau-{label}",
#                     showlegend=True,
#                     marker=dict(size=6),
#                 ),
#                 row=1,
#                 col=1,
#             )

# # T_v profiles
# for name, (y, _Tn, Tv) in profiles.items():
#     fig.add_trace(
#         go.Scatter(
#             x=Tv,
#             y=y,
#             mode="lines",
#             name=f"{name}",
#             legendgroup=name,
#             showlegend=False,
#         ),
#         row=1,
#         col=2,
#     )
#     for ref in CASE_SETTINGS[name]["references"]:
#         y_ref = ref.get("y_over_H", np.array([]))
#         x_ref = ref.get("Tv_over_Tb0", np.array([]))
#         if y_ref.size and x_ref.size:
#             label = ref.get("label", name)
#             fig.add_trace(
#                 go.Scatter(
#                     x=x_ref,
#                     y=y_ref,
#                     mode="markers",
#                     name=f"Casseau {label}",
#                     legendgroup=f"casseau-{label}",
#                     showlegend=False,
#                     marker=dict(size=6),
#                 ),
#                 row=1,
#                 col=2,
#             )

# fig.update_xaxes(title_text="T_tr / T_b0", row=1, col=1)
# fig.update_xaxes(title_text="T_v / T_b0", row=1, col=2)
# fig.update_yaxes(title_text="y / H", row=1, col=1)

# fig.update_layout(
#     width=int(PLOT_SETTINGS["figsize"][0] * 100),
#     height=int(PLOT_SETTINGS["figsize"][1] * 100),
#     legend_title_text="Cases",
# )

# fig.show()
